In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from faiss import write_index, read_index

import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer,AutoModelForCausalLM


/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import pandas as pd
df = pd.read_csv(r"D:\COMPETITIONS\RetrieverNLP\PTMTuning\input\eedi_train.csv")

ModuleNotFoundError: No module named 'datasets'

In [11]:
print(batch['query']['input_ids'].shape)  # Should be (2, 3)
batch['contents']['input_ids'].shape  # Should be (2, 2, 3) (2 samples, 2 contents each, 3 tokens each)

torch.Size([2, 3])


torch.Size([2, 2, 3])

In [5]:
content_tokens

{'input_ids': tensor([[[ 1,  2,  3],
          [ 4,  5,  6]],
 
         [[ 7,  8,  9],
          [10, 11, 12]]]),
 'attention_mask': tensor([[[1, 1, 1],
          [1, 1, 1]],
 
         [[1, 1, 1],
          [1, 1, 1]]])}

In [2]:
# MODELS_TO_TEST = {
#     "gte-Qwen2-7B-instruct": {
#         "model_name": "Alibaba-NLP/gte-Qwen2-7B-instruct",
#         "query_instruction": "",
#         "normalize": False,
#         "trust_remote_code": True,  # 必须启用
#         "model_class": "AutoModelForCausalLM"  # 指定生成式模型接口
#     }
# }
# model_config = MODELS_TO_TEST['gte-Qwen2-7B-instruct']
# model = AutoModelForCausalLM.from_pretrained(
#         model_config["model_name"],
#         trust_remote_code=True,
#         device_map="auto",
#         torch_dtype=torch.float16
#     )

In [3]:


class CFG:
    input_path = "/root/cloud/RetrieverNLP/PTM_tuning/input/"
    train_path = f"{input_path}train.csv"
    test_path  = f"{input_path}test.csv"
    misc_path  = f"{input_path}misconception_mapping.csv"
    samp_path  = f"{input_path}sample_submission.csv"
#     max_cutoff = 50 #V1
    #max_cutoff = 100 #V2
    #max_cutoff = 50 #v3
    max_cutoff = 150 #v3
    
    is_train   = False
    if is_train:
        embd_name  = "BAAI/bge-large-en-v1.5"#online
        rerank_na  = 'BAAI/bge-reranker-large'#online
    else:
        embd_name  = "/kaggle/input/bge-large-en-v1-5/bge-large-en-v1.5"#offline
        rerank_na  = "/kaggle/input/bge-reranker-large"#offline
        
    with_fineture_reranker = True
    reranker_fineture_path = "/kaggle/input/bge-reranker-ft-v2"
#     reranker_fineture_path = "/kaggle/input/bge-reranker-ft-v3"
cfg = CFG()


train                 = pd.read_csv(cfg.train_path)
misconception_mapping = pd.read_csv(cfg.misc_path)



def make_all_question_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_question_text"] = df["ConstructName"] +" " +df["QuestionText"]
    return df
train = make_all_question_text(train)


import pandas as pd

def wide_to_long(df: pd.DataFrame) -> pd.DataFrame:
    # 第一步：处理答案文本（AnswerXText）
    text_long = pd.melt(
        df,
        id_vars    = ["QuestionId", "all_question_text", "CorrectAnswer"],
        value_vars = ["AnswerAText", "AnswerBText", "AnswerCText", "AnswerDText"],
        var_name   = 'Answer',
        value_name = 'AnswerText'
    )
    # 提取答案选项字母（如 A/B/C/D）
    text_long['Answer'] = text_long['Answer'].str.replace('Answer', '').str.replace('Text', '')

    # 第二步：处理错误概念ID（MisconceptionXId）
    misconception_long = pd.melt(
        df,
        id_vars    = ["QuestionId"],
        value_vars = ["MisconceptionAId", "MisconceptionBId", "MisconceptionCId", "MisconceptionDId"],
        var_name   = 'MisconceptionAnswer',
        value_name = 'MisconceptionId'
    )
    # 提取答案选项字母（如 A/B/C/D）
    misconception_long['Answer'] = misconception_long['MisconceptionAnswer'].str.replace('Misconception', '').str.replace('Id', '')
    misconception_long = misconception_long.drop(columns=['MisconceptionAnswer'])

    # 合并两个长格式数据
    merged_long = pd.merge(
        text_long,
        misconception_long,
        on=['QuestionId', 'Answer'],
        how='left'
    )
    
    return merged_long

train_long = wide_to_long(train)

def make_all_text(df: pd.DataFrame) -> pd.DataFrame:
    df["all_text"] = df["all_question_text"] +" " +df["Answer"]
    df = df[ (df.CorrectAnswer!=df.Answer)&(df.MisconceptionId.isnull()==False) ].reset_index(drop=True)
    return df

train_long = make_all_text(train_long)









In [11]:

# ======================================================
# 配置区（按需修改）
# ======================================================
	
# bge-en-icl
MODELS_TO_TEST = {

    "BGE-large-en": {
        "model_name": "BAAI/bge-large-en-v1.5",
        "query_instruction": "Represent this sentence for searching relevant passages: ",
        "normalize": True,
        "trust_remote_code":False
    },
    "BGE-M3": {
        "model_name": "BAAI/bge-m3",
        "query_instruction": "",
        "normalize": False , # M3自带归一化
        "trust_remote_code":False
    },

    
     "jina-embeddings-v3": {
        "model_name": "jinaai/jina-embeddings-v3",
        "query_instruction": "",
        "normalize": True , 
        "trust_remote_code":True  
    },
    
    "snowflake-arctic-embed-l-v2.0": {
            "model_name": "Snowflake/snowflake-arctic-embed-l-v2.0",
            "query_instruction": "",
            "normalize": True , 
            "trust_remote_code":False  
        },
    

}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PER_GPU_BATCH_SIZE = 32

# ======================================================
# 核心评估函数
# ======================================================
import numpy as np
import pandas as pd
import torch
import faiss
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity

# 新版本加载模型代码，解决KeyError: 'mistral'问题
def evaluate_model(model_config, misconception_mapping, train_data):
    """评估单个模型的Recall@50"""
    model_name = model_config["model_name"]
    trust_remote_code = model_config.get("trust_remote_code", False)
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    PER_GPU_BATCH_SIZE = 32
    
    print(f"Loading model: {model_name}")
    
    try:
        # 尝试先判断模型类型
        if "Qwen" in model_name:
            print("识别为Qwen生成模型，使用AutoModelForCausalLM加载")
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code,
                torch_dtype=torch.float16 if "7B" in model_name else torch.float32,  # 大模型使用半精度
                device_map="auto" if torch.cuda.is_available() else None
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
        elif model_name=='snowflake-arctic-embed-l-v2.0':
            print("使用AutoModel加载嵌入模型")
            model = AutoModel.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                add_pooling_layer=False,
                trust_remote_code=trust_remote_code
            )
        else:
            # 嵌入模型默认使用AutoModel
            print("使用AutoModel加载嵌入模型")
            model = AutoModel.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
            tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                trust_remote_code=trust_remote_code
            )
        

        model = model.to(DEVICE)
        print(f"模型加载成功: {model_name}")
        
    except KeyError as e:
        if "mistral" in str(e):
            print("检测到mistral模型加载错误。尝试使用特殊处理...")
            # 针对mistral模型的特殊处理
            try:
                from transformers import MistralForCausalLM, MistralConfig
                
                print("使用MistralForCausalLM直接加载")
                model = MistralForCausalLM.from_pretrained(
                    model_name,
                    trust_remote_code=trust_remote_code,
                    torch_dtype=torch.float16,
                    device_map="auto" if torch.cuda.is_available() else None
                )
                tokenizer = AutoTokenizer.from_pretrained(
                    model_name,
                    trust_remote_code=trust_remote_code
                )
                model = model.to(DEVICE)
                print(f"Mistral模型加载成功: {model_name}")
            except Exception as mistral_error:
                print(f"Mistral特殊处理失败: {mistral_error}")
                raise
        else:
            raise
    
    # 构建候选池索引映射
    misconception_ids = misconception_mapping['MisconceptionId'].values.tolist()
    misconception_texts = misconception_mapping['MisconceptionName'].values.tolist()
    id_to_index = {_id: idx for idx, _id in enumerate(misconception_ids)}

    # 生成候选池向量
    def encode_texts(texts, is_query=False):
        all_vectors = []
        for i in tqdm(range(0, len(texts), PER_GPU_BATCH_SIZE)):
            batch_texts = texts[i:i+PER_GPU_BATCH_SIZE]
            
            # 添加指令前缀（如果需要）
            if is_query and model_config.get("query_instruction", ""):
                batch_texts = [model_config["query_instruction"] + t for t in batch_texts]
            
            # 准备输入
            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(DEVICE)
            
            # 提取嵌入向量
            with torch.no_grad():
                if isinstance(model, AutoModelForCausalLM.__bases__[0]) or "CausalLM" in model.__class__.__name__:
                    # 对于生成模型，使用隐藏状态作为嵌入
                    outputs = model(**inputs, output_hidden_states=True)
                    # 对于生成模型，使用最后一层隐藏状态的平均作为嵌入
                    embeddings = outputs.hidden_states[-1].mean(dim=1)
                else:
                    # 对于嵌入模型，使用标准方法
                    outputs = model(**inputs)
                    embeddings = outputs.last_hidden_state[:, 0]  # 使用CLS token
            
            # 归一化（如果需要）
            if model_config.get("normalize", False):
                embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
            # all_vectors.append(embeddings.cpu().numpy())
            all_vectors.append(embeddings.cpu().to(torch.float32).numpy())
        
        return np.concatenate(all_vectors, axis=0)
    
    # 编码候选池
    print("编码候选池文本...")
    sentence_embeddings = encode_texts(misconception_texts)
    
    # 编码查询
    print("编码查询文本...")
    query_texts = [item["all_text"] for _, item in train_data.iterrows()]
    ground_truth_indices = [id_to_index[item["MisconceptionId"]] for _, item in train_data.iterrows()]
    query_embeddings = encode_texts(query_texts, is_query=True)
    
    # 计算Recall@50
    def calculate_recall(sent_emb, query_emb, gt_indices, top_k=50):
        index = faiss.IndexFlatIP(sent_emb.shape[1])
        index.add(sent_emb.astype(np.float32))
        
        _, top_indices = index.search(query_emb.astype(np.float32), top_k)
        return np.mean([1 if gt in indices else 0 for gt, indices in zip(gt_indices, top_indices)])
    
    print("计算Recall@50...")
    return calculate_recall(sentence_embeddings, query_embeddings, ground_truth_indices)


# 使用示例:
# results = {}
# for model_name, config in MODELS_TO_TEST.items():
#     print(f"\n=== Evaluating {model_name} ===")
#     recall = evaluate_model(config, misconception_mapping, train_long)
#     results[model_name] = recall
#     print(f"{model_name} Recall@50: {recall*100:.2f}%")


# ======================================================
# 执行测试
# ======================================================

    # 加载数据（需替换为实际数据）
    # misconception_mapping = ... 
    # train_data = ...


results = {}
for model_name, config in MODELS_TO_TEST.items():
    print(f"\n=== Evaluating {model_name} ===")
    recall = evaluate_model(config, misconception_mapping, train_long)
    results[model_name] = recall
    print(f"{model_name} Recall@50: {recall*100:.2f}%")

# 打印最终对比结果
print("\n=== Final Results ===")
for model, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:<15} | Recall@50: {score*100:.2f}%")


=== Evaluating multilingual-e5-large ===
Loading model: intfloat/multilingual-e5-large
使用AutoModel加载嵌入模型
模型加载成功: intfloat/multilingual-e5-large
编码候选池文本...


100%|██████████| 81/81 [00:01<00:00, 42.38it/s]


编码查询文本...


100%|██████████| 137/137 [00:16<00:00,  8.51it/s]


计算Recall@50...
multilingual-e5-large Recall@50: 41.56%

=== Evaluating snowflake-arctic-embed-l-v2.0 ===
Loading model: Snowflake/snowflake-arctic-embed-l-v2.0
使用AutoModel加载嵌入模型
模型加载成功: Snowflake/snowflake-arctic-embed-l-v2.0
编码候选池文本...


100%|██████████| 81/81 [00:01<00:00, 41.37it/s]


编码查询文本...


100%|██████████| 137/137 [00:16<00:00,  8.48it/s]


计算Recall@50...
snowflake-arctic-embed-l-v2.0 Recall@50: 62.86%

=== Final Results ===
snowflake-arctic-embed-l-v2.0 | Recall@50: 62.86%
multilingual-e5-large | Recall@50: 41.56%


In [ ]:
# === Final Results ===
# BGE-large-en    | Recall@50: 64.51%
# BGE-M3          | Recall@50: 59.04%
# jina-embeddings-v3 Recall@50: 67.12%
# multilingual-e5-large Recall@50: 41.56%
# snowflake-arctic-embed-l-v2.0 | Recall@50: 62.86%

import gzip
import json
import pandas as pd

def load_json_gz_to_dataframe(file_path: str) -> pd.DataFrame:
    """
    从 .json.gz 文件中读取每行一个 JSON 对象，转换为 pandas DataFrame。

    参数:
        file_path: 压缩 JSON 文件路径

    返回:
        pandas DataFrame
    """
    data = []
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    obj = json.loads(line)
                    data.append(obj)
                except json.JSONDecodeError:
                    continue  # 跳过格式错误的行

    return pd.DataFrame(data)

# 示例用法
df = load_json_gz_to_dataframe('../input/json_pack_full.json.gz')
print(df.head())
index=False)

  query_id  content_id                                        explanation  \
0      0_D        1672  Correct calculation: 3×(2+4)-5 = 3×6-5 = 18-5 ...   
1   1000_A         891  Correct Calculation:\n\( \frac{1-t}{t-1} = \fr...   
2   1000_D         353  Correct calculation:\n\[\n\frac{1-t}{t-1} = \f...   
3   1002_B        1715  To convert \( \frac{3}{20} \) to a decimal, th...   
4   1002_C        2308  Correct calculation: \( 3 \div 20 = 0.15 \).  ...   

                    timestamp error  
0  2025-04-20T22:00:24.440664  None  
1  2025-04-20T22:00:24.440664  None  
2  2025-04-20T22:00:24.440664  None  
3  2025-04-21T21:23:04.159306  None  
4  2025-04-20T22:00:24.440664  None  


In [46]:
import pandas as pd 

DATAPATH = '../input/negatives_hard.csv'
negatives_hard = pd.read_csv(DATAPATH)  
print(negatives_hard.iloc[0].keys())
negatives_hard.iloc[0].values

DATAPATH = '../input/eedi_content.csv'
eedi_content = pd.read_csv(DATAPATH)  
print(eedi_content.iloc[0].keys())
eedi_content.iloc[0].values

Index(['query_id', 'content_id', 'hard_negatives', 'hard_negative_count'], dtype='object')
Index(['MisconceptionId', 'MisconceptionName'], dtype='object')


array([0, 'Does not know that angles in a triangle sum to 180 degrees'],
      dtype=object)

In [48]:
negatives_hard

,query_id,content_id,hard_negatives,hard_negative_count
0,84_A,1568,"2301,642,2127,2113,1144,1304,2080,287,1285,1139",10
1,1591_A,2450,"2197,1788,2353,192,305,2388,648,396,867,910",10
2,885_A,1877,"226,926,551,1007,2355,561,2394,1851,1370,465",10
3,59_C,2206,"1601,1313,247,1965,2191,914,1710,2023,2112,531",10
4,1193_C,2362,"309,2121,100,48,2117,356,793,1707,725,2334",10
...,...,...,...,...
2970,587_A,706,"15,2306,1507,1005,1963,1516,2140,561,987,1999",10
2971,355_D,926,"2355,1877,1007,1687,561,1399,1370,1791,515,2161",10
2972,983_C,234,"1332,2414,1514,997,1887,2093,1648,1219,835,1978",10
2973,1306_C,2275,"118,1014,1873,836,2416,745,761,2427,1543,1340",10


In [ ]:
negatives_hard.groupby("query_id")["hard_negatives"].apply(lambda x: x.split(",")).to_dict()

AttributeError: 'Series' object has no attribute 'split'

In [39]:
eedi_content['MisconceptionId'] = eedi_content['MisconceptionId'].astype(str)
eedi_content_dict = eedi_content.set_index('MisconceptionId')['MisconceptionName'].to_dict()

def get_content_of_hardnage_byquery_id(query_id, eedi_content_dict, negatives_hard):
    """
    根据查询ID获取相关的hardneg内容
    :param query_id: 查询ID
    :param eedi_content: eedi_content DataFrame
    :param negatives_hard: hardnegatives DataFrame
    :return: 相关的hardneg内容
    """
    # 获取与查询ID匹配的hardneg内容
    # row_to_dict
    if query_id not in negatives_hard['query_id'].values:
        print("当前的query_id在hardnegatives中不存在")
        return []
    query_row_to_dict = negatives_hard[negatives_hard['query_id'] == query_id].to_dict(orient='records')[0]
    current_content_id = query_row_to_dict['content_id']
    
    # 获取hardneg内容的ID列表
    hardneg_ids = query_row_to_dict['hard_negatives'].split(",")
    
    # 根据ID从eedi_content中获取对应的内容
    related_contents = [eedi_content_dict.get(hardneg_id.strip(), '未获取到相关内容') for hardneg_id in hardneg_ids]
    current_contents = eedi_content_dict.get(str(current_content_id), '未获取到相关内容')
    return related_contents,current_contents




ModuleNotFoundError: No module named 'sklearn'

In [45]:
!pip install sklearn

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [43]:
qid = '885_A'
related_contents,current_content = get_content_of_hardnage_byquery_id(qid, eedi_content_dict, negatives_hard)
print('当前content内容:', current_content)
for i, content in enumerate(related_contents):
    print(f"Related content {i+1}: {content}")

当前content内容: Thinks rectangles have rotational symmetry or order 4
Related content 1: Thinks rectangles have line symmetry in the diagonals 
Related content 2: Believes the number of sides gives the order of rotational symmetry
Related content 3: Believes a rectangle has four equal sides
Related content 4: Does not connect rotational symmetry to number of sides/vertices
Related content 5: Does not know how to find order of rotational symmetry
Related content 6: Misinterprets the order of division in worded problems
Related content 7: Does not know the properties of an equilateral triangle
Related content 8: Does not know the properties of an isosceles triangle
Related content 9: Does not believe a shape can have two correct names e.g. rectangle and parallelogram
Related content 10: Does not know the diagonals of a square bisect at right angles


In [ ]:
# !pip install -U huggingface_hub
# !export HF_ENDPOINT=https://hf-mirror.com

# !pip install einops
# !rm -rf ~/.cache/huggingface/transformers_modules/
